# Charter School NlP Fun Stuff

## Install relevant packages

In [1]:
#basic processing
import pandas as pd
import numpy as np 
#networking and scraping
import requests
from bs4 import BeautifulSoup
#text processing
import spacy
# import spacy_transformers #not really usable because of problems with cuda
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
# bring out the nuclear weapons for this job
import os
import openai
#openai.api_key = os.getenv("openai")
# nltk.download('stopwords')
# nltk.download("puntk")
#nltk.data.path.append("C://Users//chris//AppData//Roaming//nltk_data")
from dotenv import load_dotenv
load_dotenv()
import Levenshtein
import ast


In [2]:
# Load the data
input_data=pd.read_csv('data/active_charter_schools_report.csv')
# The columns are just ugly, so let's clean them up
new_columns = input_data.columns.str.replace("-","").str.replace(" ","").str.lower() 
input_data.columns = new_columns

In [3]:
input_data.columns

Index(['countycode', 'countydescription', 'lea', 'schoolname',
       'officialschoolname', 'principal/directornameprefix',
       'principal/directorfirstname', 'principal/directormiddlename',
       'principal/directorlastname', 'principal/directornamesuffix',
       'principal/directoremail', 'boardchairnameprefix',
       'boardchairfirstname', 'boardchairmiddlename', 'boardchairlastname',
       'boardchairnamesuffix', 'boardchairemail', 'addressline1',
       'addressline2', 'city', 'state', 'zipcode5', 'mailingaddressline1',
       'mailingaddressline2', 'mailingcity', 'mailingstate', 'mailingzipcode5',
       'phone', 'fax', 'openingeffectivedate', 'gradelevelcurrent',
       'gradelevelapproved', 'virtualstatus', 'ueiregistrationnumber',
       'samexpirationdate'],
      dtype='object')

In [4]:
input_data.head()

,countycode,countydescription,lea,schoolname,officialschoolname,principal/directornameprefix,principal/directorfirstname,principal/directormiddlename,principal/directorlastname,principal/directornamesuffix,...,mailingstate,mailingzipcode5,phone,fax,openingeffectivedate,gradelevelcurrent,gradelevelapproved,virtualstatus,ueiregistrationnumber,samexpirationdate
0,32,Durham,00A,North Carolina Cyber Academy,North Carolina Cyber Academy,Mr,Martez,-,Hill,-,...,-,-,984.262.9237,919.406.0063,07/01/2015,KG:01:02:03:04:05:06:07:08:09:10:11:12,KG:01:02:03:04:05:06:07:08:09:10:11:12,FULLVIRTUAL,CMMJNNC4MV61,04/01/2023
1,32,Durham,00B,NC Virtual Academy,NC Virtual Academy,-,Katharine,-,Gomersall,-,...,-,-,919.346.0121,919.324.6597,07/01/2015,KG:01:02:03:04:05:06:07:08:09:10:11:12,KG:01:02:03:04:05:06:07:08:09:10:11:12,FULLVIRTUAL,LKVLXN6LFDH5,09/26/2023
2,1,Alamance,01B,River Mill Academy,River Mill Academy,Mr,Jeffrey,-,Dishmon,-,...,-,-,336.229.0909,336.228.9941,06/30/1998,KG:01:02:03:04:05:06:07:08:09:10:11:12,KG:01:02:03:04:05:06:07:08:09:10:11:12,NOTVIRTUAL,H2Y8LJYM6NF7,02/24/2023
3,1,Alamance,01C,Clover Garden,Clover Garden,Mrs,Heather,-,Vereyken,-,...,-,-,336.586.9440,336.586.9477,06/30/2001,KG:01:02:03:04:05:06:07:08:09:10:11:12,KG:01:02:03:04:05:06:07:08:09:10:11:12,NOTVIRTUAL,NMLRVLDWFBW3,08/31/2023
4,1,Alamance,01D,The Hawbridge School,The Hawbridge School,Ms,Jennifer,-,Shelton,-,...,NC,27340,336.376.1122,336.376.6996,07/01/2004,KG:01:02:03:04:05:06:07:08:09:10:11:12,KG:01:02:03:04:05:06:07:08:09:10:11:12,NOTVIRTUAL,KJ5MXKD978A9,01/13/2023


In [5]:
input_data['website']=input_data['principal/directoremail'].str.split('@').str[-1]
#list of columns I care about
charter_school_column=['countydescription','schoolname','principal/directoremail','website']

In [6]:
input_data[charter_school_column].head(15)

,countydescription,schoolname,principal/directoremail,website
0,Durham,North Carolina Cyber Academy,mhill@myncca.com,myncca.com
1,Durham,NC Virtual Academy,kgomersall@ncvacademy.org,ncvacademy.org
2,Alamance,River Mill Academy,jdishmon@rivermill-academy.org,rivermill-academy.org
3,Alamance,Clover Garden,heathervereyken@cgsnc.org,cgsnc.org
4,Alamance,The Hawbridge School,jshelton@hawbridgeschool.org,hawbridgeschool.org
5,Alamance,Alamance Community School,lpaynter@alamancecommunityschool.net,alamancecommunityschool.net
6,Avery,Marjorie Williams Academy,caustin@williamsacademy.org,williamsacademy.org
7,Beaufort,Washington Montessori,darla.prescott@wmpcs.org,wmpcs.org
8,Bladen,Paul R. Brown Leadership Academy,jwray@prbla.org,prbla.org
9,Bladen,Emereau: Bladen,elizabethcole@emereau.org,emereau.org


## Search up board of directors of each google link and get ready to parse the data

In [7]:
import requests
from bs4 import BeautifulSoup

## search for the board of directors page from site

def get_google_search_results_boards(school_name: str):
    # set the school name to search for
    # create a Google search query URL
    query_url = f"https://www.google.com/search?q={school_name}+meet+the+board+of+directors"
    # make a request to the query URL and get the HTML content
    response = requests.get(query_url)
    html_content = response.text
    # parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')
    # find all the search result links on the page
    search_result_links = soup.find_all('a')
    # extract the URLs of the first search result link from each search engine
    result_urls = []
    for link in search_result_links:
        href = link.get('href')
        if href.startswith('/url?q='):
            result_url = href.split('/url?q=')[1].split('&')[0]
            result_urls.append(result_url)
        if len(result_urls) == 1:
            break

    # print the resulting URLs
    return(result_urls)


In [8]:
input_data['board_of_directors_link']=input_data['schoolname'].apply(get_google_search_results_boards)

In [9]:
feature_columns=['countydescription','schoolname','principal/directoremail','zipcode5','boardchairfirstname','boardchairlastname','website','board_of_directors_link']
input_data[feature_columns].to_csv('data/checkdata.csv',index=False)
#checkpoint read input data
#input_data = pd.read_csv('data/checkdata.csv')

## Getting feature google snippets are problematic

In [10]:
# import requests
# from bs4 import BeautifulSoup

# def get_google_feature_snippet(url):
#     # Send a GET request to the URL and retrieve the HTML content
#     response = requests.get(url)
#     html = response.text
    
#     # Parse the HTML content using BeautifulSoup
#     soup = BeautifulSoup(html, 'html.parser')
    
#     # Find the Google feature snippet element, if one exists
#     feature_snippet = soup.find('div', class_='kp-header')
    
#     # If a feature snippet was found, extract its text content and return it
#     if feature_snippet:
#         return feature_snippet.text.strip()
#     else:
#         return None

## Add function that would determine if link is part of original domain

In [11]:
from urllib.parse import urlparse

def compare_domains(url1, url2):
    domain1 = urlparse(url1).netloc.replace("www.", "")
    domain2 = urlparse(url2).netloc.replace("www.", "")
    return domain1 == domain2

In [12]:
input_data['link_domain_match']=input_data.apply(lambda x: compare_domains(f"https://{x['website']}",x['board_of_directors_link'][0]),axis=1)

In [13]:
input_data[['link_domain_match','board_of_directors_link','website']].sample(4)

,link_domain_match,board_of_directors_link,website
130,True,[https://scstemacademy.org/board-of-directors/],scstemacademy.org
41,True,[https://www.voyageracademy.net/board-of-direc...,voyageracademy.net
45,False,[https://www.dpi.nc.gov/documents/fbs/charters...,reachingallminds.com
123,False,[https://unityclassical.cfacademy.school/schoo...,unityclassical.org


In [14]:
input_data['link_domain_match'].value_counts()

True     142
False     64
Name: link_domain_match, dtype: int64

## Add data scrapper function for per string - some of these functions are used to test the data scrapper and whos the best

In [15]:
# loop through each URL and scrape the text content
def website_text_content(url: str):
    # make a request to the URL and get the HTML content
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
    response = requests.get(url, headers=headers)
    html_content = response.text
    # parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')
    # remove the header and footer from the HTML content
    for tag in soup(['header', 'footer']):
        tag.decompose()
    # extract the text content from the HTML
    text_content = soup.get_text().replace('\n', ' ').replace('\r', ' ').replace('\t', ' ').replace('.', ' ')

    #stop_words = set(stopwords.words('english'))
    #tokens = nltk.word_tokenize(text_content)
    #filtered_tokens = [token.lower() for token in tokens if token.lower() not in stop_words and len(token) > 1]
    #filtered_text = ' '.join(filtered_tokens)
    
    # remove extra spaces and line breaks
    text_content = re.sub(r"\s+", " ", text_content).strip()
    # add spaces between words that are capitalized
    text_content = re.sub(r'(?<!^)(?=[A-Z])', ' ', text_content)
    # remove punctuation
    words = text_content.split()
    # remove single word characters because of middle initials
    filtered_words = [word for word in words if len(word) > 1]
    filtered_text = ' '.join(filtered_words)
    
    return filtered_text
    # print the text content


In [16]:
def scrape_middle_content(url:str):
    try:
        headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
        # Fetch the content from the URL
        response = requests.get(url,headers=headers)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while fetching the URL: {e}")
        return None

    # Parse the HTML content
    soup = BeautifulSoup(response.content, "html.parser")

    # Find the main content container
    main_container = soup.find("main") or soup.find("article") or soup.find("div", {"role": "main"})

    if main_container is None:
        print("Could not find the main content container.")
        return None

    # Find the middle content
    content_elements = main_container.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'ul', 'ol', 'li', 'blockquote'])

    if not content_elements:
        print("Could not find any content elements.")
        return None

    middle_index = len(content_elements) // 2
    middle_content = content_elements[middle_index]

    return middle_content

In [65]:
def extract_main_content(url):
    try:
        headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
        response = requests.get(url,headers=headers)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while fetching the URL: {e}")
        return None

    soup = BeautifulSoup(response.content, "html.parser")

    # Remove unwanted elements
    for tag in ['header', 'footer', 'nav']:
        elements_to_remove = soup.find_all(tag)
        for elem in elements_to_remove:
            elem.decompose()

    # Extract text from the remaining elements
    text_content = ' '.join(soup.stripped_strings)
    # remove extra spaces and line breaks
    text_content = re.sub(r"\s+", " ", text_content).strip()
    # add spaces between words that are capitalized
    text_content = re.sub(r'(?<!^)(?=[A-Z])', ' ', text_content)
    # remove punctuation
    words = text_content.split()
    # remove single word characters because of middle initials
    filtered_words = [word for word in words if len(word) > 1]
    filtered_text = ' '.join(filtered_words)

    return filtered_text

In [100]:
def clean_string(s: str):
    # Remove digits
    s = re.sub(r'\d+', '', s)
    # Remove special characters
    s = re.sub(r'[^\w\s]', '', s)
    # Remove non-readable characters
    s = ''.join(filter(lambda x: x.isprintable(), s))
    return s

import string

def remove_punctuation_and_single_chars(text):
    # Remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))

    # Remove single character words
    words = text.split()
    filtered_words = [word for word in words if len(word) > 1]
    result = ' '.join(filtered_words)

    return result

In [31]:
input_data['board_of_directors_link']=input_data['board_of_directors_link'].astype(str)
input_data['board_of_directors_link']=input_data['board_of_directors_link'].str.strip("[]").str.strip("''")

In [66]:
input_data['director_text_content']=input_data['board_of_directors_link'].apply(extract_main_content)

An error occurred while fetching the URL: 404 Client Error: Not Found for url: https://www.artspacecharter.org/apps/pages/index.jsp%3FuREC_ID%3D2012305%26type%3Dd


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


An error occurred while fetching the URL: 403 Client Error: Forbidden for url: https://www.cabarruscharter.org/apps/pages/index.jsp%3FuREC_ID%3D362974%26type%3Dd%26termREC_ID%3D%26pREC_ID%3DSPOT-12273
An error occurred while fetching the URL: 403 Client Error: Forbidden for url: https://www.aceacademycharter.org/apps/pages/index.jsp%3FuREC_ID%3D471307%26type%3Dd
An error occurred while fetching the URL: 403 Client Error: Forbidden for url: https://www.concordlakeacademy.org/m/pages/index.jsp%3FuREC_ID%3D369004%26type%3Dd
An error occurred while fetching the URL: 403 Client Error: Forbidden for url: https://www.davidsoncharter.org/m/pages/index.jsp%3FuREC_ID%3D432636%26type%3Dd


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


An error occurred while fetching the URL: 403 Client Error: Forbidden for url: https://www.necprepschool.com/apps/pages/index.jsp%3FuREC_ID%3D485812%26type%3Dd
An error occurred while fetching the URL: 404 Client Error: Not Found for url: https://www.youngsvilleacademy.org/apps/pages/index.jsp%3FuREC_ID%3D450318%26type%3Dd%26pREC_ID%3D970870
An error occurred while fetching the URL: 404 Client Error: Not Found for url: https://www.micharter.org/apps/pages/index.jsp%3FuREC_ID%3D229679%26type%3Dd%26pREC_ID%3D523176
An error occurred while fetching the URL: 403 Client Error: Forbidden for url: https://www.guilfordprep.org/m/pages/index.jsp%3FuREC_ID%3D458156%26type%3Dd%26
An error occurred while fetching the URL: 403 Client Error: Forbidden for url: https://piedmontclassical.com/apps/pages/index.jsp%3FuREC_ID%3D440921%26type%3Dd


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


An error occurred while fetching the URL: 403 Client Error: Forbidden for url: https://www.langtreecharter.org/m/pages/index.jsp%3FuREC_ID%3D365741%26type%3Dd
An error occurred while fetching the URL: 403 Client Error: Forbidden for url: https://www.iredellcharter.org/apps/pages/index.jsp%3FuREC_ID%3D382301%26type%3Dd
An error occurred while fetching the URL: 403 Client Error: Forbidden for url: https://www.wlakeprep.org/apps/pages/index.jsp%3FuREC_ID%3D382511%26type%3Dd
An error occurred while fetching the URL: 403 Client Error: Forbidden for url: https://www.scholarsacademy.org/m/pages/index.jsp%3FuREC_ID%3D334971%26type%3Dd
An error occurred while fetching the URL: 404 Client Error: Not Found for url: https://www.charlottesecondary.org/board-members
An error occurred while fetching the URL: 403 Client Error: Forbidden for url: https://www.charlottechoicecharter.org/m/pages/index.jsp%3FuREC_ID%3D518162%26type%3Dd
An error occurred while fetching the URL: 404 Client Error: Not Found f

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


An error occurred while fetching the URL: 403 Client Error: Forbidden for url: https://www.wilsonpreparatoryacademy.org/apps/pages/index.jsp%3FuREC_ID%3D389167%26type%3Dd


In [3]:
#input_data.to_csv('check_data_point2.csv',escapechar='\\', index=False)
input_data = pd.read_csv('check_data_point2.csv')

## Refine function to find multiple people - you should really try to experiment with different models because its really annoying to find people.

In [17]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

def extract_names_bert(text):
    model_name = "dbmdz/bert-large-cased-finetuned-conll03-english"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForTokenClassification.from_pretrained(model_name)

    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)

    predictions = torch.argmax(outputs.logits, dim=-1)
    tokens = tokenizer.convert_ids_to_tokens(inputs.input_ids[0])

    names = []
    current_name_tokens = []
    for token, label_id in zip(tokens, predictions[0]):
        label = model.config.id2label[label_id.item()]
        
        if label == "B-PER":
            if current_name_tokens:
                names.append(tokenizer.convert_tokens_to_string(current_name_tokens))
            current_name_tokens = [token]
        elif label == "I-PER":
            current_name_tokens.append(token)
        else:
            if current_name_tokens:
                names.append(tokenizer.convert_tokens_to_string(current_name_tokens))
                current_name_tokens = []

    if current_name_tokens:
        names.append(tokenizer.convert_tokens_to_string(current_name_tokens))

    # Extract first and last names
    
    first_last_names = []
    for name in names:
        name_parts = name.split()
        if len(name_parts) >= 2:
            first_last_names.append((name_parts[0], name_parts[-1]))
    
    first_last_names=list(set(first_last_names))

    return first_last_names

## Spacy Models are useless for NER stick to pretrain bert models

In [6]:
def find_names_sm(text_content: str):
    nlp = spacy.load("en_core_web_sm") # you can change the model to bigger ones like "en_core_web_trf" - problematic with cuda and spacy_transformers
    # process the text with the NER model
    doc = nlp(text_content)
    # extract the named entities (people's names)
    names = [ent.text for ent in doc.ents if ent.label_ == "PERSON"]
    # drop duplicates
    names=list(set(names))
    # print the names
    return names

In [7]:
def find_names_md(text_content: str):
    nlp = spacy.load("en_core_web_md") # you can change the model to bigger ones like "en_core_web_trf" - problematic with cuda and spacy_transformers
    # process the text with the NER model
    doc = nlp(text_content)
    # extract the named entities (people's names)
    names = [ent.text for ent in doc.ents if ent.label_ == "PERSON"]
    # Drop Duplicates
    names=list(set(names))
    # print the names
    return names

In [8]:
def find_names_lg(text_content: str):
    nlp = spacy.load("en_core_web_lg") # you can change the model to bigger ones like "en_core_web_trf" - problematic with cuda and spacy_transformers
    # process the text with the NER model
    doc = nlp(text_content)
    # extract the named entities (people's names)
    names = [ent.text for ent in doc.ents if ent.label_ == "PERSON"]
    # Drop Duplicates
    names=list(set(names))
    # print the names
    return names

In [10]:
#input_data['director_text_content'][40]
(input_data['director_text_content'][90])

'Board of Directors Neuse Charter School NCS Home Enrollment K-5 Tours 6-12 Tours News Information C-19 Dashboard About Neuse Charter > NCS- A Cinderella Story Strategic Plan Accountability > Testing Transparency Accountability Documents McKinney-Vento Act Title IX USDA Statement Academics > Lower Academy > Lower Academy Staff Upper Academy > Upper Academy Staff Enrichments > Electives and Enhancements Staff Performing Arts Exceptional Children > Eligibility NCDPI Exceptional Children Laws and Policies Parent Resources Exceptional Children Staff Guidance AIG Title 1 College and Career Counseling Supply Lists Operations > Student Services Documents > Parent-Student Handbook School Documents Board Documents Videos > NCS YouTube Channel NCS Board YouTube Channel Operations Staff Board of Directors Medical Information Athletics Athletic Booster Club Employment Volunteer for Neuse Calendar Connect Say Something Anonymous Reporting NCS Community Folder Resources > HomeBase NCS Clever Student

In [12]:
input_data['director_text_content'] = input_data['director_text_content'].astype(str)
input_data['director_first_last']=input_data['director_text_content'].apply(extract_names_bert) # 10 minutes to complete

In [13]:
## practically useless
input_data['director_names_sm']=input_data['director_text_content'].apply(find_names_sm)
input_data['director_names_md']=input_data['director_text_content'].apply(find_names_md)
input_data['director_names_lg']=input_data['director_text_content'].apply(find_names_lg)

In [14]:
input_data.columns

Index(['countycode', 'countydescription', 'lea', 'schoolname',
       'officialschoolname', 'principal/directornameprefix',
       'principal/directorfirstname', 'principal/directormiddlename',
       'principal/directorlastname', 'principal/directornamesuffix',
       'principal/directoremail', 'boardchairnameprefix',
       'boardchairfirstname', 'boardchairmiddlename', 'boardchairlastname',
       'boardchairnamesuffix', 'boardchairemail', 'addressline1',
       'addressline2', 'city', 'state', 'zipcode5', 'mailingaddressline1',
       'mailingaddressline2', 'mailingcity', 'mailingstate', 'mailingzipcode5',
       'phone', 'fax', 'openingeffectivedate', 'gradelevelcurrent',
       'gradelevelapproved', 'virtualstatus', 'ueiregistrationnumber',
       'samexpirationdate', 'website', 'board_of_directors_link',
       'link_domain_match', 'director_text_content', 'director_first_last',
       'director_names_sm', 'director_names_md', 'director_names_lg'],
      dtype='object')

In [15]:
charter_school_column=['lea','zipcode5','countydescription','schoolname','principal/directoremail','website','board_of_directors_link','link_domain_match','director_first_last']

In [16]:
input_data[charter_school_column].to_csv('check_data_point3.csv',escapechar='\\', index=False)

## Nuclear option goooodbye monies

10$ of my money is gone

In [ ]:
def find_names_openai(text_content:str):
  
  response = openai.Completion.create(
    model="text-davinci-003",
    prompt=f"context: {text_content} \n\n question: Who are the board members with first and last names?, give it to me as a python list If none are available give me an empty list.",
    temperature=0.86,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
  )
  return(response.choices[0].text)

# Define function to chunk a long text into smaller pieces

def chunk_text(text, chunk_size=5000):
    """Chunk a string into a list of strings with a maximum size of chunk_size."""
    chunks = []
    start = 0
    end = chunk_size
    while start < len(text):
        if end >= len(text):
            end = len(text)
        chunk = text[start:end]
        chunks.append(chunk)
        start += chunk_size
        end += chunk_size
    return chunks

def find_names_in_chunks(text_chunks:list):
  
  results = []
  
  for chunk in text_chunks:
    names = find_names_openai(chunk)
    results.append(names)
  
  return results

In [ ]:
input_data['clean_director_text_content']= input_data['director_text_content'].apply(clean_string)
input_data['text_chunks'] = input_data['clean_director_text_content'].apply(chunk_text)
#input_data['director_names_openai']=input_data['text_chunks'].apply(find_names_in_chunks) # that was 10$ 

In [ ]:
#input_data.to_csv('check_data_point4.csv',escapechar='\\', index=False)
input_data = pd.read_csv('check_data_point4.csv')

In [ ]:
## checkpoint marker
# charter_school_column=['countydescription','schoolname','principal/directoremail','website','director_names_openai']
# input_data[charter_school_column].to_csv('director_name.csv',index=False)

In [ ]:
def openai_cleaner(input_string:str):
    input_string = input_string.replace('\'','')
    input_string = input_string.replace('[','')
    input_string = input_string.replace(']','')
    input_string = input_string.replace('\"','')
    input_string = input_string.replace("NA", "")
    input_string = input_string.replace("\n", "")
    input_string = input_string.replace("\\\\n\\\\n", "")
    input_string=input_string.replace("(", "").replace(")", "")
    input_string=input_string.replace("\\\\n", "")
    input_string=input_string.replace("Answer:", "")
    input_string = re.sub(r'(\n[\s]*)+', ' ', input_string)
    # Remove patterns such as \n\nAnswer:, \n\n(, ), and (list)
    patterns = ['Answer:', '\(\w+\)', '\(list\)']
    for pattern in patterns:
        input_string = re.sub(r'\n\n{}[\s]*'.format(pattern), ' ', input_string)
    # Remove leading and trailing whitespaces
    input_string = input_string.strip()
    words_to_remove = ['Director', 'Assistant', 'Parent Seat','Parent', 'Seat', 'Community', 'Board', 'Treasurer','Alumni','-','Business Analyst']
    
    for word in words_to_remove:
        input_string = input_string.replace(word, "")
    
    return input_string 

# def second_cleaner(input_string:list):
#     words_to_remove = ['Director', 'Assistant', 'Parent Seat','Parent', 'Seat', 'Community', 'Board', 'Treasurer','Alumni','-']
#     for word in words_to_remove:
#         input_string = input_string.str.replace(word, '', regex=False)
#     return input_string

# def remove_words(words_list: list):
#     remove_list = ['Director', 'Assistant', 'Parent Seat','Parent', 'Seat', 'Community', 'Board', 'Treasurer','Alumni','-']
#     return [word for word in words_list if word not in remove_list]

In [ ]:
input_data['director_names_openai_clean']=input_data['director_names_openai'].apply(openai_cleaner)
input_data['cleaner_names'] = input_data['director_names_openai_clean'].apply(find_names_lg)

In [ ]:
input_data['cleaner_names']

In [ ]:
input_data['cleaner_names'][10]

In [ ]:
# len(input_data[input_data['cleaner_names'] == '[]' ])
def count_empty_lists(lst):
    return len(lst) == 0
# apply the function to each element of the column and count the number of empty lists
num_empty_lists = input_data['cleaner_names'].apply(count_empty_lists).sum()
print(num_empty_lists) 

In [ ]:
input_data.columns

### 74 schools that dont have a list of board of directors

In [ ]:
input_data[['schoolname','officialschoolname','zipcode5','Hyperlink','cleaner_names']].to_csv('names_cleaning.csv',index=False)

## Bert Model has a problem with black names :(

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = "My name is Sarah Crofer and I live in Berlin"

ner_results = nlp(example)
names = []
this_name = []
all_names_list_tmp = []

for ner_dict in ner_results:
    if ner_dict['entity'] == 'B-PER':
        if len(this_name) == 0:
            this_name.append(ner_dict['word'])
        else:
            all_names_list_tmp.append([this_name])
            this_name = []
            this_name.append(ner_dict['word'])
    elif ner_dict['entity'] == 'I-PER':
        this_name.append(ner_dict['word'])

all_names_list_tmp.append([this_name])

print(all_names_list_tmp)

final_name_list = []
for name_list in all_names_list_tmp:
    full_name = ' '.join(name_list[0]).replace(' ##', '').replace(' .', '.')
    final_name_list.append([full_name])

print(final_name_list)

# for result in ner_results:
#     if result['entity'] == 'B-PER' or result['entity'] == 'I-PER':
#         names.append(result['word'])

#print(names)

## Coding archive, we probably could use this to make it better next time

In [ ]:
from transformers import BertTokenizer, BertModel
import torch
import spacy

# Load the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-large-cased')
model = BertModel.from_pretrained('bert-large-cased')

# Load the spaCy model for named entity recognition
nlp = spacy.load("en_core_web_md")

# Define the text to analyze
text = test_content

# Tokenize the text and convert to PyTorch tensors
tokens = tokenizer.tokenize(text)
tokens = ['[CLS]'] + tokens + ['[SEP]']
token_ids = tokenizer.convert_tokens_to_ids(tokens)
token_tensor = torch.tensor([token_ids])

# Run the text through the BERT model
with torch.no_grad():
    last_hidden_states = model(token_tensor)[0]

# Extract the named entities using spaCy
named_entities = []
doc = nlp(text)
for ent in doc.ents:
    if ent.label_ == "PERSON":
        named_entities.append(ent.text)

print(named_entities)

In [ ]:
#holy shit finding peoples names are difficult
## refine people's names and seperate by first and last name - find likely voter registration affilation.

## Voter Lookup 

In [2]:
state_voter=pd.read_csv('data/ncvoter_Statewide.txt',sep="\t", header=0,encoding='ISO-8859-1')
#input_data_copy = pd.read_excel('data/check_data_point3.xlsx')

C:\Users\chris\AppData\Local\Temp\ipykernel_18244\2955062259.py:1: DtypeWarning: Columns (23,34,35,36,37,38,39,41,42,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,65,66) have mixed types. Specify dtype option on import or set low_memory=False.
  state_voter=pd.read_csv('data/ncvoter_Statewide.txt',sep="\t", header=0,encoding='ISO-8859-1')


In [3]:
state_voter.columns

Index(['county_id', 'county_desc', 'voter_reg_num', 'ncid', 'last_name',
       'first_name', 'middle_name', 'name_suffix_lbl', 'status_cd',
       'voter_status_desc', 'reason_cd', 'voter_status_reason_desc',
       'res_street_address', 'res_city_desc', 'state_cd', 'zip_code',
       'mail_addr1', 'mail_addr2', 'mail_addr3', 'mail_addr4', 'mail_city',
       'mail_state', 'mail_zipcode', 'full_phone_number', 'confidential_ind',
       'registr_dt', 'race_code', 'ethnic_code', 'party_cd', 'gender_code',
       'birth_year', 'age_at_year_end', 'birth_state', 'drivers_lic',
       'precinct_abbrv', 'precinct_desc', 'municipality_abbrv',
       'municipality_desc', 'ward_abbrv', 'ward_desc', 'cong_dist_abbrv',
       'super_court_abbrv', 'judic_dist_abbrv', 'nc_senate_abbrv',
       'nc_house_abbrv', 'county_commiss_abbrv', 'county_commiss_desc',
       'township_abbrv', 'township_desc', 'school_dist_abbrv',
       'school_dist_desc', 'fire_dist_abbrv', 'fire_dist_desc',
       'water_di

In [4]:
state_voter[['voter_reg_num','voter_status_desc','first_name','last_name','birth_year','zip_code','race_code','ethnic_code','party_cd','gender_code']]

,voter_reg_num,voter_status_desc,first_name,last_name,birth_year,zip_code,race_code,ethnic_code,party_cd,gender_code
0,9005990,ACTIVE,RUTH,AABEL,1935,27253.0,W,NL,UNA,F
1,9178574,REMOVED,JONI,AARDEN,1978,NaN,W,UN,UNA,F
2,9205561,ACTIVE,TIMOTHY,AARMSTRONG,1966,27302.0,W,UN,REP,M
3,9048723,ACTIVE,CHRISTINA,AARON,1976,27215.0,W,UN,UNA,F
4,9019674,ACTIVE,CLAUDIA,AARON,1945,27215.0,W,NL,UNA,F
...,...,...,...,...,...,...,...,...,...,...
8248669,26095,ACTIVE,MICHELLE,ZITNEY,1998,28714.0,W,NL,UNA,F
8248670,15223,ACTIVE,VICTORIA,ZITNEY,1966,28714.0,W,NL,UNA,F
8248671,26014,ACTIVE,DAWN,ZULICK,1963,28714.0,W,NL,REP,F
8248672,26085,ACTIVE,PAMELA,ZULLINGER,1955,28714.0,W,NL,UNA,F


In [5]:
state_voter=state_voter.loc[(state_voter['voter_status_desc'] == 'ACTIVE') & (state_voter['zip_code'].notna())]

In [6]:
state_voter[(state_voter['first_name']=='CARY') & (state_voter['last_name']=='CAIN')]

,county_id,county_desc,voter_reg_num,ncid,last_name,first_name,middle_name,name_suffix_lbl,status_cd,voter_status_desc,...,sanit_dist_abbrv,sanit_dist_desc,rescue_dist_abbrv,rescue_dist_desc,munic_dist_abbrv,munic_dist_desc,dist_1_abbrv,dist_1_desc,vtd_abbrv,vtd_desc
2032628,32,DURHAM,30017641,BL254918,CAIN,CARY,DONALD,II,A,ACTIVE,...,NaN,NaN,NaN,NaN,DUR,DURHAM,16.0,16TH PROSECUTORIAL,45,45


In [7]:
state_voter=state_voter.dropna(subset=['first_name','last_name'], how='all')

In [8]:
state_voter[['first_name','last_name']]=state_voter[['first_name','last_name']].astype(str)

In [9]:
state_voter['full_name'] = state_voter[['first_name','last_name']].apply(lambda x: ' '.join(x), axis=1)

## zipcode calculator

## Levenshtein distance lexical name similarity

In [10]:
def lexical_similarity(name, name_column):
    name_column = name_column.apply(lambda x: Levenshtein.ratio(name, x))
    return name_column

In [120]:
input_data_copy.columns

Index(['countydescription', 'schoolname', 'principal/directoremail', 'website',
       'board_of_directors_link', 'link_domain_match', 'director_first_last',
       'cleaner_names', 'zipcode', 'political_affilation', 'postal_code',
       'latitude', 'longitude'],
      dtype='object')

In [121]:
index_number = 3
print(input_data['schoolname'][index_number])
print(input_data['zipcode5'][index_number])
input_data_copy['cleaner_names'][3]

NameError: name 'input_data' is not defined

In [11]:
file_path = "data/US.txt"
us_zip = pd.read_table(file_path, header=None)
us_zip.columns = ["country_code","postal_code", "place_name", "admin_name1", "admin_code1",
              "admin_name2", "admin_code2", "admin_name3", "admin_code3",
              "latitude", "longitude", "accuracy"]
us_zip['postal_code'] = us_zip['postal_code'].apply(lambda x : str(x).zfill(5))
state_voter['zip_code']=state_voter['zip_code'].astype(int)
state_voter['zip_code']=state_voter['zip_code'].astype(str)
state_voter=pd.merge(state_voter,us_zip[['postal_code', 'latitude', 'longitude']],left_on='zip_code',right_on='postal_code',how='left') 

In [248]:
# name_to_compare = 'BRANDON RUSSELL'
# similarity_scores = lexical_similarity(name_to_compare, state_voter['full_name'])
# state_voter['Similarity'] = similarity_scores
# parse_zip=state_voter[['full_name','party_cd','Similarity','zip_code','latitude','longitude']].sort_values(by='Similarity', ascending=False).head(50)
# parse_zip.reset_index(inplace=True)

In [12]:
from geopy.distance import geodesic

def calculate_distance(lat, lon, lat_column, lon_column):
    distance_column = []
    for lat2, lon2 in zip(lat_column, lon_column):
        distance = geodesic((lat, lon), (lat2, lon2)).miles
        distance_column.append(distance)
    return pd.Series(distance_column)

# # Reference coordinates (latitude, longitude)
# zipcode = '27217'
# ref_lat = us_zip['latitude'].loc[us_zip['postal_code'] == zipcode].iloc[0]
# ref_lon = us_zip['longitude'].loc[us_zip['postal_code'] == zipcode].iloc[0]

# # Calculate distances and add as a new column in the DataFrame
# parse_zip['Distance']=calculate_distance(ref_lat, ref_lon, parse_zip['latitude'], parse_zip['longitude'])

#parse_zip

In [260]:
# board_party_affilation=parse_zip.sort_values(by = ['Similarity', 'Distance'], ascending = [False,True]).head(1)['party_cd'].values.tolist()
# party_list =[]
# party_list.append(board_party_affilation[0])
# party_list

In [135]:
# take note of this issue
state_voter[state_voter["latitude"].isnull() | state_voter["longitude"].isnull()]['zip_code']

3195775    28703
3195778    28703
3197691    28703
3197693    28703
4210144    27838
4213796    27838
4215174    27838
4215175    27838
4218121    27838
4218136    27838
Name: zip_code, dtype: object

In [132]:
us_zip[us_zip.postal_code == '27838']

,country_code,postal_code,place_name,admin_name1,admin_code1,admin_name2,admin_code2,admin_name3,admin_code3,latitude,longitude,accuracy


In [21]:
def find_political_affilation(list_names:list,lat,lon):
    party_list =[]
    for names in list_names:
        name_to_compare = names.upper()
        #print(name_to_compare)
        similarity_scores = lexical_similarity(name_to_compare, state_voter['full_name'])
        state_voter['Similarity'] = similarity_scores
        parse_zip=state_voter[['full_name','party_cd','Similarity','zip_code','latitude','longitude']].sort_values(by='Similarity', ascending=False).head(50)
        parse_zip.reset_index(inplace=True)
        ref_lat = lat
        ref_lon = lon
        parse_zip['Distance']=calculate_distance(ref_lat, ref_lon, parse_zip['latitude'], parse_zip['longitude'])
        board_party_affilation=parse_zip.sort_values(by = ['Similarity', 'Distance'], ascending = [False,True]).head(1)['party_cd'].values.tolist()
        party_list.append(board_party_affilation[0])
    return party_list

In [ ]:
import heapq
from typing import List, Tuple
#gpt4 political affiliation help on how to do this
def optimize_find_political_affiliation(list_names: List[str], lat: float, lon: float) -> List[str]:
    party_list = []
    for name in list_names:
        name = name.upper()
        scores = []
        for i, voter_name in enumerate(state_voter['full_name']):
            similarity = lexical_similarity(name, voter_name)
            if len(scores) < 50:
                # If we don't have 50 scores yet, just add it to the heap.
                heapq.heappush(scores, (similarity, i))
            else:
                # If we already have 50 scores, add the new one and remove the smallest.
                heapq.heappushpop(scores, (similarity, i))
        # Get the indices of the top 50 most similar names.
        top_indices = [i for _, i in scores]
        # Calculate distances for these 50 records.
        distances = calculate_distance(lat, lon, state_voter.loc[top_indices, 'latitude'], state_voter.loc[top_indices, 'longitude'])
        # Pair up similarities and distances, then sort by similarity first and distance second.
        sorted_records = sorted(zip(scores, distances), key=lambda x: (-x[0][0], x[1]))
        # Get the political affiliation of the top record.
        top_record_index = sorted_records[0][0][1]
        top_party = state_voter.loc[top_record_index, 'party_cd']
        party_list.append(top_party)
    return party_list

In [17]:
input_data_copy=pd.read_csv('data/final_checkpoint.csv',encoding='latin-1')
## data preprocessing after reading
input_data_copy['zipcode']=input_data_copy['zipcode'].astype(str)
input_data_copy['cleaner_names']=input_data_copy['cleaner_names'].apply(ast.literal_eval)
#input_data_copy=pd.merge(input_data_copy,us_zip[['postal_code', 'latitude', 'longitude']],left_on='zipcode',right_on='postal_code',how='left') 


In [22]:
input_data_copy['political_affilation']=input_data_copy.apply(lambda x: find_political_affilation(x['cleaner_names'],x['latitude'],x['longitude']),axis=1)

In [23]:
input_data_copy.to_csv('data/final_checkpoint_clean.csv',index=False)

In [329]:
find_political_affilation(input_data_copy_test['cleaner_names'].apply(ast.literal_eval)[0],input_data_copy_test['zipcode'][0])

Julie Marks
Jill Hammergren
Debora Williams
Audrey Martin Mc Coy  Vice
Becky Taylor Dee Ray
Angela Howard


['DEM', 'REP', 'UNA', 'UNA', 'UNA', 'UNA']

In [33]:
def count_dem(lst):
    return lst.count('DEM')
def count_rep(lst):
    return lst.count('REP')
def count_una(lst):
    return lst.count('UNA')

In [36]:
input_data_copy['count_dem']=input_data_copy['political_affilation'].apply(count_dem)
input_data_copy['count_rep']=input_data_copy['political_affilation'].apply(count_rep)
input_data_copy['count_una']=input_data_copy['political_affilation'].apply(count_una)

In [37]:
input_data_copy.head(5)

,countydescription,schoolname,principal/directoremail,website,board_of_directors_link,link_domain_match,director_first_last,cleaner_names,zipcode,political_affilation,postal_code,latitude,longitude,count_dem,count_rep,count_una
0,Durham,North Carolina Cyber Academy,mhill@myncca.com,myncca.com,https://www.myncca.com/our-school-board,True,"[('Jill', 'Hammergren'), ('Audrey', 'Martin'),...","[Julie Marks, Jill Hammergren, Debora Williams...",27713,"[DEM, DEM, REP, DEM, REP, DEM, DEM]",27713,35.9112,-78.9178,5,2,0
1,Durham,NC Virtual Academy,kgomersall@ncvacademy.org,ncvacademy.org,https://ncva.k12.com/about-our-school/board-of...,False,"[('Sara', 'W'), ('C.', 'Struhs'), ('Sara', 'St...","[Steven Peters, Sara Struhs, Mark Fleming, Wal...",27713,"[UNA, REP, DEM, DEM, UNA, REP]",27713,35.9112,-78.9178,2,2,2
2,Alamance,River Mill Academy,jdishmon@rivermill-academy.org,rivermill-academy.org,http://www.rivermill-academy.org/about_r_m_a/b...,True,[],"[James Penrod, Jill Davis, Dean Martin Brinkle...",27253,"[DEM, UNA, DEM, UNA, DEM, UNA, DEM, UNA, UNA, ...",27253,36.0310,-79.3814,4,1,5
3,Alamance,Clover Garden,heathervereyken@cgsnc.org,cgsnc.org,http://cgsnc.org/home/grizzly-life/board-of-di...,True,"[('Cathy', 'June'), ('Chad', 'Huffines'), ('Am...","[Chad Huffines, Amy Murrell, Randi Russell, As...",27217,"[UNA, REP, UNA, REP, UNA, UNA, REP]",27217,36.1288,-79.4114,0,3,4
4,Alamance,The Hawbridge School,jshelton@hawbridgeschool.org,hawbridgeschool.org,http://thehawbridgeschool.org/board/,False,[],"[April Williams, Renee Lynch, Nick Harrison, A...",27340,"[DEM, DEM, DEM, REP, DEM, UNA, DEM, REP]",27340,35.9488,-79.3297,5,2,1


In [44]:
input_data_copy.sort_values(by = ['count_dem'], ascending = [False]).head(30)

,countydescription,schoolname,principal/directoremail,website,board_of_directors_link,link_domain_match,director_first_last,cleaner_names,zipcode,political_affilation,postal_code,latitude,longitude,count_dem,count_rep,count_una
12,Buncombe,Evergreen Community Charter,susan.mertz@evergreenccs.org,evergreenccs.org,https://www.evergreenccs.org/board,True,"[('Alicia', 'Cole'), ('Susan', 'Mertz'), ('Sus...","[Susan Mertz, Alicia Cole, Ehren Cruz, Rick Br...",28805,"[UNA, DEM, DEM, UNA, UNA, UNA, DEM, DEM, UNA, ...",28805,35.6004,-82.4918,10,1,6
63,Granville,Oxford Preparatory,swannerab@oxfordprep.org,oxfordprep.org,https://www.oxfordprep.org/Governing-Board,True,[],"[Chance Wilkinson, Jason Pegram, Andrew Swanne...",27565,"[REP, REP, DEM, DEM, DEM, DEM, DEM, DEM, DEM, ...",27565,36.3313,-78.6134,9,3,2
113,Mecklenburg,Commonwealth High,Jonathan.Kay@als-education.com,als-education.com,https://commonwealthhs.com/apps/pages/index.js...,False,[],"[Michael Drake, Sheila Burke, Joseph Betancour...",28205,"[DEM, REP, UNA, REP, DEM, DEM, DEM, DEM, DEM, ...",28205,35.2200,-80.7881,9,3,2
40,Durham,Central Park School For Child,john@cpsfc.org,cpsfc.org,https://cpscnc.org/our-board-of-trustees,False,"[('Aliyah', 'Rahman'), ('Marcia', 'Brooks'), (...","[Aliyah Abdur Rahman, Vicky Patton, Nish Evans...",27701,"[DEM, DEM, DEM, REP, DEM, DEM, DEM, DEM, DEM, ...",27701,35.9967,-78.8966,9,1,0
50,Forsyth,Quality Education Academy,qualityeducation@qeschools.org,qeschools.org,https://www.qeschools.org/our-board,True,[],"[Tamara Turner, Jefferson Caruthers, Jr., Adol...",27105,"[DEM, DEM, DEM, DEM, DEM, DEM, DEM, UNA, UNA, ...",27105,36.1440,-80.2376,8,0,2
188,Wake,Envision Science Academy,cfuller@envisionscienceacademy.com,envisionscienceacademy.com,https://www.esaeagles.com/about-esa/board-of-d...,False,"[('Ryan', 'Cotterman'), ('Roland', 'Lewis'), (...","[Esperanza Montessori, Hassan Kingsberry Esq, ...",27587,"[UNA, DEM, DEM, DEM, DEM, DEM, DEM, DEM, DEM, ...",27587,35.9815,-78.5392,8,0,3
17,Buncombe,Francine Delany New School,buffy@fdnsc.net,fdnsc.net,https://fdnsc.net/our-school/board-of-directors/,True,"[('Katrina', 'Froehlich'), ('Missy', 'Reed'), ...","[Katrina Oliver, Elana Froehlich, Missy Reed, ...",28806,"[UNA, DEM, DEM, REP, DEM, UNA, UNA, DEM, UNA, ...",28806,35.5808,-82.6078,7,1,4
117,Mecklenburg,Stewart Creek High,Tamara.Wynn@als-education.com,als-education.com,https://www.stewartcreekhs.com/apps/pages/inde...,False,[],"[Reginald Pincham, Morgan Kelly Cohen, Nichole...",28208,"[DEM, REP, DEM, UNA, UNA, DEM, DEM, DEM, DEM, ...",28208,35.2358,-80.8964,7,1,2
178,Wake,Raleigh Charter High School,lhuddleston@raleighcharterhs.org,raleighcharterhs.org,https://www.raleighcharterhs.org/board-of-dire...,True,"[('David', 'Ennis'), ('Jonathan', 'Green'), ('...","[Jonathan Green, Sandra Headen, DJ Hill, Lisa ...",27605,"[UNA, DEM, REP, UNA, DEM, UNA, DEM, UNA, UNA, ...",27605,35.7908,-78.6530,7,1,6
100,Mecklenburg,Sugar Creek Charter,Turner.Cheryl@thesugarcreek.org,thesugarcreek.org,https://thesugarcreek.org/about/board,True,"[('H.', 'Ives'), ('Matthew', 'Tilley'), ('Anth...","[Anthony Daise, Bryan Ives Chair, Daye Brake, ...",28206,"[DEM, UNA, DEM, DEM, REP, UNA, UNA, DEM, REP, ...",28206,35.2522,-80.8265,7,3,7


In [40]:
input_data_copy.to_csv('data/final_checkpoint_clean.csv',index=False)